# LIBRERIAS

In [1]:
import pandas as pd
import os

# LECTURA

In [2]:
path = os.getcwd()
file = "VISA Reglas Intercambio V25.xlsx"
file_path = os.path.join(path, file).replace("\\", "/")

In [3]:
df = pd.read_excel(file_path, skiprows=3)

# Preprocesamiento

In [ ]:
## NO USAR ##
# Conversion del tipo de datos "object" a "str" 
#columns_object = df.select_dtypes(include="object").columns
#df[columns_object] = df[columns_object].astype(str)

In [4]:
customer_code = str(input("Ingresar codigo de cliente legacy: ")).upper()
jurisdiction_count = int(input("Ingresar cantidad de jurisdicciones a generar: "))
jurisdiction_values = []

for i in range(0, jurisdiction_count):
    value = str(input(f"Ingresar jurisdiccion #'{i+1}: "))
    jurisdiction_values.append(value.upper())

print(customer_code)
print(jurisdiction_values)

BRDRO
['RO', '5', '9']


In [5]:
df_filter = df[
    (df["REGION_COUNTRY_CODE"].astype(str).isin(jurisdiction_values)) & (df["VALID_UNTIL"].isnull())
]

# Diccionarios

In [7]:
# Creación de diccionario para jurisdicciones
jurisdiction_dictionary = (
    df.drop_duplicates(subset="REGION_COUNTRY_CODE")
    .set_index("REGION_COUNTRY_CODE")["JURISDICTION"]
    .to_dict()
)

jurisdiction_dictionary = {
    str(k): str(v) for k, v in jurisdiction_dictionary.items() if not pd.isna(k)
}
jurisdiction_dictionary


{'TT': 'Trinidad and Tobago',
 'RO': 'Romania',
 'PE': 'Peru',
 '3': 'LAC Intraregional',
 '9': 'Interregional',
 'HU': 'Hungary',
 'HN': 'Honduras',
 'GR': 'Greece',
 '5': 'Europe Intraregional',
 'DO': 'Dominican Republic',
 'KY': 'Cayman Islands',
 'BS': 'Bahamas',
 'AW': 'Aruba',
 'CW,SX': 'Dutch Area',
 'BB': 'Barbados',
 'ZA': 'South Africa',
 '7': 'CEMEA Intraregional',
 'JM': 'Jamaica'}

In [8]:
# Creación de diccionario para tablas en plataforma legacy
table_dictionary = {
    "ERSTHU": "TAVIERSTHU.dbo.TMP_TRX",
    "BRDRO": "TAVIBRDRO_CLC.DBO.TMP_TRX",
}
table_dictionary

{'ERSTHU': 'TAVIERSTHU.dbo.TMP_TRX', 'BRDRO': 'TAVIBRDRO_CLC.DBO.TMP_TRX'}

# Creación de funciones

In [58]:
# Creación de funciones reutilizables

from numpy import append


def parse_operator_unique(value: str):
    if value.startswith("NOT:"):
        return "<>", value[4:].strip()
    return "=", value.strip()


def parse_operator_multiple(value: str):
    if value.startswith("NOT:"):
        return "NOT IN", value[4:].strip()
    return "IN", value.strip()


def split_str_values(value: str):
    return ",".join([f"'{v.strip()}'" for v in value.split(",")])


def split_int_values(value: str):
    return ",".join([f"{v.strip()}" for v in value.split(",")])


def replace_space(value: str):
    values = value.replace("''Space'","'Space'").replace("Space"," ").strip()
    return values


def replace_blank(value: str):
    values = value.replace("BLANK","").replace("Blank","").strip()
    return values


def split_range_values(column: str, value: str):
    is_not = value.startswith("NOT:")
    values = value.replace("NOT:", "").strip()

    ranges = []
    single_values = []

    for val in values.split(","):
        val = val.strip()
        if "-" in val:
            start, end = val.split("-")
            ranges.append(
                f"({column} {'NOT BETWEEN' if is_not else 'BETWEEN'} {start} AND {end})"
            )
        else:
            single_values.append(val)

    in_condition = ""
    if single_values:
        operator = "NOT IN" if is_not else "IN"
        in_condition = f"{column} {operator} ({', '.join(single_values)})"

    logical_connector = "AND" if is_not else "OR"
    conditions = ranges + ([in_condition] if in_condition else [])

    return f" ({f' {logical_connector} '.join(conditions)}) "


In [10]:
def create_joins(column: str, value: str):
    match column:
        case "BUSINESS_MODE":
            print("hola")

In [79]:
from logging import handlers
from re import split


def create_where(column: str, value: str):
    def business_mode(value):
        if value == "Acquiring":
            return "T1.[VSS_BMOD_ID] = 1"
        elif value == "Issuing":
            return "T1.[VSS_BMOD_ID] = 2"

    def issuer_country(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"CTRY.CTRY_DESC {operator} ({split_values})"

    def issuer_region(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"CTRY.CTRY_DESC {operator} ({split_values})"

    def technology_indicator(value):
        operator, clean_value = parse_operator_unique(value)
        return f"technlogy_indicator {operator} '{clean_value}'"

    def product_id(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"P.BIN_PRD_CD {operator} ({split_values})"

    def fast_funds(value):
        operator, clean_value = parse_operator_unique(value)
        return f"FAST_FUND {operator} '{clean_value}'"

    def travel_indicator(value):
        operator, clean_value = parse_operator_unique(value)
        return f"TRAVEL_INDICATOR {operator} '{clean_value}'"

    def b2b_program_id(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"B2B_PROGRAM_ID {operator} ({split_values})"

    def account_funding_source(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"FUNDING_SOURCE {operator} ({split_values})"

    def nnss_indicator(value):
        operator, clean_value = parse_operator_unique(value)
        replaced_value = replace_space(clean_value)
        return f"NNSS_INDICATOR {operator} '{replaced_value}'"

    def product_subtype(value):
        operator, clean_value = parse_operator_unique(value)
        return f"PRODUCT_SUBTYPE {operator} '{clean_value}'"

    def transaction_code(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"TRANSACTION_CODE {operator} ({split_values})"

    def transaction_code_qualifier(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_int_values(clean_value)
        return f"TRANSACTION_CODE_QUALIFIER {operator} ({split_values})"

    def issuer_bin_8(value):
        range_values = split_range_values("ISSUER_BIN", value)
        return f"{range_values}"

    def acquirer_bin(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_int_values(clean_value)
        return f"ACQUIRER_BIN {operator} ({split_values})"

    def acquirer_business_id(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_int_values(clean_value)
        return f"ACQUIRER_BUSINESS_ID {operator} ({split_values})"

    def transaction_amount_currency(value):
        operator, clean_value = parse_operator_unique(value)
        return f"CURRENCY_ID {operator} '{clean_value}'"

    def transaction_amount(value):
        operator, clean_value = parse_operator_unique(value)
        if operator == "=":
            operator = ""
        return f"V_AMT{operator} {clean_value}"

    def acquirer_country(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"ACQ_CTRY {operator} ({split_values})"

    def acquirer_region(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"ACQ_REG {operator} ({split_values})"

    def merchant_country_code(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"MCT_CTRY {operator} ({split_values})"

    def merchant_country_region(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"MCT_REGION {operator} ({split_values})"

    def merchant_category_code(value):
        range_values = split_range_values("MCC", value)
        return f"{range_values}"

    def requested_payment_service(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_blank(split_values)
        return f"REQUEST_PAYMENT_SERVICES {operator} ({replaced_value})"

    def usage_code(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_blank(split_values)
        return f"USAGE_CODE {operator} ({replaced_value})"

    def authorization_characteristics_indicator(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"AUTH_IND {operator} ({split_values})"

    def authorization_code(value):
        if value =='NOT:Valid':
            return "(RIGHT(T0.AUT_CD,1)='X' OR LEFT(T0.AUT_CD,3)='SVC' OR RIGHT(T0.AUT_CD,5)='     ' OR RIGHT(T0.AUT_CD,5)='00000' OR RIGHT(T0.AUT_CD,5)='0000 ' OR RIGHT(T0.AUT_CD,5)='0000N' OR RIGHT(T0.AUT_CD,5)='0000P' OR RIGHT(T0.AUT_CD,5)='0000Y')"
        else:
            return "(RIGHT(T0.AUT_CD,1)<>'X' AND LEFT(T0.AUT_CD,3)<>'SVC' AND RIGHT(T0.AUT_CD,5)<>'     ' AND RIGHT(T0.AUT_CD,5)<>'00000' AND RIGHT(T0.AUT_CD,5)<>'0000 ' AND RIGHT(T0.AUT_CD,5)<>'0000N' AND RIGHT(T0.AUT_CD,5)<>'0000P' AND RIGHT(T0.AUT_CD,5)<>'0000Y')"

    def pos_terminal_capability(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_space(split_values)
        return f"POS_TERMINAL {operator} ({replaced_value})"

    def cardholder_id_method(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_space(split_values)
        return f"CARDHOLDER_ID_METHOD {operator} ({replaced_value})"

    def pos_entry_mode(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_space(split_values)
        return f"POS_ENTRY_MODE {operator} ({replaced_value})"

    def timeliness(value):
        return "Hola"

    def reimbursement_attribute(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"REIMBURSEMENT_ATTRIBUTE {operator} ({split_values})"

    def special_condition_indicator(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"SPECIAL_CONDITION_INDICATOR {operator} ({split_values})"

    def fee_program_indicator(value):
        operator, clean_value = parse_operator_unique(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_blank(split_values)
        return f"FEE_PROGRAM_INDICATOR {operator} {replaced_value}"

    def moto_eci_indicator(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_space(split_values)
        return f"MOTO_ECI_INDICATOR {operator} ({replaced_value})"

    def acceptance_terminal_indicator(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_space(split_values)
        return f"acceptance_terminal_indicator {operator} ({replaced_value})"

    def prepaid_card_indicator(value):
        operator, clean_value = parse_operator_unique(value)
        split_values = split_str_values(clean_value)
        return f"SPECIAL_CONDITION_INDICATOR {operator} {split_values}"

    def pos_environment_code(value):
        operator, clean_value = parse_operator_unique(value)
        split_values = split_str_values(clean_value)
        return f"POS_ENVIRONMENT_CODE {operator} {split_values}"

    def business_format_code(value):
        operator, clean_value = parse_operator_unique(value)
        split_values = split_str_values(clean_value)
        return f"BUSINESS_FORMAT_CODE {operator} {split_values}"

    def business_application_id(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_blank(split_values)
        return f"BUSINESS_APPLICATION_ID {operator} ({replaced_value})"

    def type_purchase(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_int_values(clean_value)
        return f"TYPE_PURCHASE {operator} ({split_values})"

    def network_identification_code(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_blank(split_values)
        return f"NETWORK_IDENTIFICATION_CODE {operator} ({replaced_value})"

    def message_reason_code(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"MSG_REA_COD {operator} ({split_values})"

    def surcharge_amount(value):
        operator, clean_value = parse_operator_unique(value)
        if clean_value == ">0":
            operator, clean_value = ">=", "0"
        return f"SURCHARGE_AMOUNT {operator} {clean_value}"

    def authorized_amount(value):
        return ""

    def authorization_response_code(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replaced_value = replace_blank(split_values)
        return f"AUTHORIZATION_RESPONSE_CODE {operator} ({replaced_value})"

    def merchant_verification_value(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"MCT_VER_VAL {operator} ({split_values})"

    def dynamic_currency_indicator(value):
        operator, clean_value = parse_operator_unique(value)
        split_values = split_str_values(clean_value)
        return f"DCC_IND_ID {operator} {split_values}"

    def cvv2_result_code(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"CVV2_RSLT_ID {operator} ({split_values})"

    def national_tax_indicator(value):
        operator, clean_value = parse_operator_unique(value)
        split_values = split_str_values(clean_value)
        return f"NTL_TAX_ICL {operator} {split_values}"

    def merchant_vat(value):
        operator, clean_value = parse_operator_unique(value)
        split_values = split_str_values(clean_value)
        replace_value=replace_blank(split_values)
        return f"NTL_TAX_ICL {operator} {replace_value}"

    def summary_commodity(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"SMY_CMY_CD {operator} ({split_values})"

    def processing_code_transaction_type(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        replace_value=replace_blank(split_values)
        return f"PROCESSING_CODE_TRANSACTION_TYPE {operator} ({replace_value})"

    def point_of_service_condition_code(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"POINT_OF_SERVICE_CONDITION_CODE {operator} ({split_values})"

    handlers = {
        "BUSINESS_MODE": business_mode,
        "ISSUER_COUNTRY": issuer_country,
        "ISSUER_REGION": issuer_region,
        "TECHNOLOGY_INDICATOR": technology_indicator,
        "PRODUCT_ID": product_id,
        "FAST_FUNDS": fast_funds,
        "TRAVEL_INDICATOR": travel_indicator,
        "B2B_PROGRAM_ID": b2b_program_id,
        "ACCOUNT_FUNDING_SOURCE": account_funding_source,
        "NNSS_INDICATOR": nnss_indicator,
        "PRODUCT_SUBTYPE": product_subtype,
        "TRANSACTION_CODE": transaction_code,
        "TRANSACTION_CODE_QUALIFIER": transaction_code_qualifier,
        "ISSUER_BIN_8": issuer_bin_8,
        "ACQUIRER_BIN": acquirer_bin,
        "ACQUIRER_BUSINESS_ID": acquirer_business_id,
        "TRANSACTION_AMOUNT_CURRENCY": transaction_amount_currency,
        "TRANSACTION_AMOUNT": transaction_amount,
        "ACQUIRER_COUNTRY": acquirer_country,
        "ACQUIRER_REGION": acquirer_region,
        "MERCHANT_COUNTRY_CODE": merchant_country_code,
        "MERCHANT_COUNTRY_REGION": merchant_country_region,
        "MERCHANT_CATEGORY_CODE": merchant_category_code,
        "REQUESTED_PAYMENT_SERVICE": requested_payment_service,
        "USAGE_CODE": usage_code,
        "AUTHORIZATION_CHARACTERISTICS_INDICATOR": authorization_characteristics_indicator,
        "AUTHORIZATION_CODE": authorization_code,
        "POS_TERMINAL_CAPABILITY": pos_terminal_capability,
        "CARDHOLDER_ID_METHOD": cardholder_id_method,
        "POS_ENTRY_MODE": pos_entry_mode,
        "TIMELINESS": timeliness,
        "REIMBURSEMENT_ATTRIBUTE": reimbursement_attribute,
        "SPECIAL_CONDITION_INDICATOR": special_condition_indicator,
        "FEE_PROGRAM_INDICATOR": fee_program_indicator,
        "MOTO_ECI_INDICATOR": moto_eci_indicator,
        "ACCEPTANCE_TERMINAL_INDICATOR": acceptance_terminal_indicator,
        "PREPAID_CARD_INDICATOR": prepaid_card_indicator,
        "POS_ENVIRONMENT_CODE": pos_environment_code,
        "BUSINESS_FORMAT_CODE": business_format_code,
        "BUSINESS_APPLICATION_ID": business_application_id,
        "TYPE_PURCHASE": type_purchase,
        "NETWORK_IDENTIFICATION_CODE": network_identification_code,
        "MESSAGE_REASON_CODE": message_reason_code,
        "SURCHARGE_AMOUNT": surcharge_amount,
        "AUTHORIZED_AMOUNT": authorized_amount,
        "AUTHORIZATION_RESPONSE_CODE": authorization_response_code,
        "MERCHANT_VERIFICATION_VALUE": merchant_verification_value,
        "DYNAMIC_CURRENCY_INDICATOR": dynamic_currency_indicator,
        "CVV2_RESULT_CODE": cvv2_result_code,
        "NATIONAL_TAX_INDICATOR": national_tax_indicator,
        "MERCHANT_VAT": merchant_vat,
        "SUMMARY_COMMODITY": summary_commodity,
        "PROCESSING_CODE_TRANSACTION_TYPE": processing_code_transaction_type,
        "POINT_OF_SERVICE_CONDITION_CODE": point_of_service_condition_code,
    }

    if column in handlers:
        return handlers[column](value)
    else:
        raise KeyError(f"Unsupported column: {column}")

In [76]:
create_where("POINT_OF_SERVICE_CONDITION_CODE","NOT:8")


"POINT_OF_SERVICE_CONDITION_CODE NOT IN ('8')"

In [18]:
from pandas import DataFrame


def generate_updates(df: DataFrame, table_name: str, jurisdiction: str):
    query_updates = []
    columns = [df.columns[i] for i in range(17, 71)]

    for _, row in df.iterrows():
        conditions = []
        joins = []
        for column in columns:
            row_value = row[column]
            if pd.isna(row_value):
                pass
            else:
                value = create_where(column,str(row_value))
                #print(f"{column}:{value}")
                conditions.append(f"{value}")

        update_value = int(row["INTELICA_ID"])
        query = (
            f"UPDATE T1 SET REI_CRI_ID = {update_value} FROM {table_name} T1 "
            + " LEFT JOIN ".join(joins)
            + f"WHERE T1.TRX_SCP_ID = '{jurisdiction}' AND "
            + " AND ".join(conditions)
            + ";"
        )
        query_updates.append(query)



    return query_updates

In [77]:
generate_updates(df_filter,"TAERSTHU_RPT.DBO.TMP_TRX","5")


["UPDATE T1 SET REI_CRI_ID = 1 FROM TAERSTHU_RPT.DBO.TMP_TRX T1 WHERE T1.TRX_SCP_ID = '5' AND CTRY.CTRY_DESC IN ('US') AND FUNDING_SOURCE IN ('D','P') AND TRANSACTION_CODE IN ('05','06','25','26') AND TRANSACTION_CODE_QUALIFIER IN (0,1) AND MCT_CTRY IN ('US') AND USAGE_CODE IN ('1') AND PROCESSING_CODE_TRANSACTION_TYPE NOT IN ('26');",
 "UPDATE T1 SET REI_CRI_ID = 2 FROM TAERSTHU_RPT.DBO.TMP_TRX T1 WHERE T1.TRX_SCP_ID = '5' AND CTRY.CTRY_DESC IN ('PR','VI') AND FUNDING_SOURCE IN ('D','P') AND TRANSACTION_CODE IN ('05','06','25','26') AND TRANSACTION_CODE_QUALIFIER IN (0,1) AND MCT_CTRY IN ('US') AND USAGE_CODE IN ('1') AND PROCESSING_CODE_TRANSACTION_TYPE NOT IN ('26');",
 "UPDATE T1 SET REI_CRI_ID = 3 FROM TAERSTHU_RPT.DBO.TMP_TRX T1 WHERE T1.TRX_SCP_ID = '5' AND CTRY.CTRY_DESC IN ('AS','GU','MH','MP','PW','UM') AND FUNDING_SOURCE IN ('D','P') AND TRANSACTION_CODE IN ('05','06','25','26') AND TRANSACTION_CODE_QUALIFIER IN (0,1) AND MCT_CTRY IN ('US') AND USAGE_CODE IN ('1') AND PROCES

In [14]:
from typing import List


def export_to_sql_file(query_updates: List[str], file_name: str):
    with open(file_name, "w", encoding="utf-8") as file:
        for query in query_updates:
            file.write(query + "\n")
    print(f"SQL queries exported to {file_name}")

In [15]:
updates_list = []
for i in jurisdiction_values:
    df_filter_jurisdiction = df_filter[df_filter["REGION_COUNTRY_CODE"].astype(str) == i]
    updates_list = generate_updates(
        df_filter_jurisdiction, table_dictionary[customer_code], i
    )
    file_name = str(jurisdiction_dictionary[i]+".sql")
    export_to_sql_file(updates_list,file_name)

SQL queries exported to Romania.sql
SQL queries exported to Europe Intraregional.sql
SQL queries exported to Interregional.sql


In [ ]:
legacy_column_rename = {
    "BUSINESS_MODE": "VSS_BMOD_ID",
    "ISSUER_COUNTRY": "ARG_CTRY_ID",
    "ISSUER_REGION": "ARG_REG_ID",
    "TECHNOLOGY_INDICATOR": "",
    "PRODUCT_ID": "",
    "FAST_FUNDS": "",
    "TRAVEL_INDICATOR": "",
    "B2B_PROGRAM_ID": "",
    "ACCOUNT_FUNDING_SOURCE": "",
    "NNSS_INDICATOR": "",
    "PRODUCT_SUBTYPE": "",
    "TRANSACTION_CODE": "",
    "TRANSACTION_CODE_QUALIFIER": "",
    "ISSUER_BIN_8": "",
    "ACQUIRER_BIN": "",
    "ACQUIRER_BUSINESS_ID": "",
    "TRANSACTION_AMOUNT_CURRENCY": "",
    "TRANSACTION_AMOUNT": "",
    "ACQUIRER_COUNTRY": "",
    "ACQUIRER_REGION": "",
    "MERCHANT_COUNTRY_CODE": "",
    "MERCHANT_COUNTRY_REGION": "",
    "MERCHANT_CATEOGORY_CODE": "",
    "REQUESTED_PAYMENT_SERVICE": "",
    "USAGE_CODE": "",
    "AUTHORIZATION_CHARACTERISTICS_INDICATOR": "",
    "AUTHORIZATION_CODE": "",
    "POS_TERMINAL_CAPABILITY": "",
    "CARDHOLDER_ID_METHOD": "",
    "POS_ENTRE_MODE": "",
    "TIMELINESS": "",
    "REIMBURSEMENT_ATTRIBUTE": "",
    "SPECIAL_CONDITION_INDICATOR": "",
    "FEE_PROGRAM_INDICATOR": "",
    "MOTO_ECI_INDICATOR": "",
    "ACCEPTANCE_TERMINAL_INDICATOR": "",
    "PREPAID_CARD_INDICATOR": "",
    "POS_ENVIRONMENT_CODE": "",
    "BUSINESS_FORMAT_CODE": "",
    "BUSINESS_APPLICATION_ID": "",
    "TYPE_PURCHASE": "",
    "NETWORK_IDENTIFICATION_CODE": "",
    "MESSAGE_REASON_CODE": "",
    "SURCHARGE_AMOUNT": "",
    "AUTHORIZED_AMOUNT": "",
    "AUTHORIZATION_RESPONSE_CODE": "",
    "MERCHANT_VERIFICATION_VALUE": "",
    "DYNAMIC_CURRENCY_INDICATOR": "",
    "CVV2_RESULT_CODE": "",
    "NATIONAL_TAX_INDICATOR": "",
    "MERCHANT_VAT": "",
    "SUMMARY_COMMODITY": "",
    "PROCESSING_CODE_TRANSACTION_TYPE": "",
    "POINT_OF_SERVICE_CONDITION_CODE": "",
}

In [ ]:
columnas = [(index, col_name) for index, col_name in enumerate(df_filter.columns)]
columnas